<a href="https://colab.research.google.com/github/jerryzhu1/Have-fun-with-deep-learning----pytorch-tensorflow/blob/master/Word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import glob
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-beta1
# !pip install tensorflow==2.0.0-beta1
import pickle

import tensorflow as tf
import pandas as pd
import re
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')

def clean_text(text):
    text = str(text).lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join(porter.stem(word) for word in text.split() if word not in stop_words) 
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17825944863658724207
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14781377401161120269
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9977661843109918919
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14892338381
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15815407596330777298
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
df = []
for f in glob.glob('/content/drive/My Drive/yelp_data/*'):
    df_tmp = pd.read_csv(f)
    df.append(df_tmp)
df_all = pd.concat(df)

In [0]:
df_all.head()
# df_all = df_all.head(10000)

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemmatize_text,stem_text,snowball_stem_text,text_clean
0,0,VLZPqnPatkdsz-w9aTGFNA,0,2014-06-15 23:14:17,0,KtX-oVw_tW7OM933iQulBw,5.0,literally wouldnt able find one negative thing...,12,S0b8_tFfOJjfERP4_bPMpQ,literally wouldnt able find one negative thing...,liter wouldnt abl find one neg thing say dr ma...,liter wouldnt abl find one negat thing say dr ...,liter wouldnt abl find one neg thing say dr ma...
1,1,cMFe-S47UGetMkEXz5dhKQ,0,2016-11-26 16:10:10,0,I93niSj-jLMVAfjHV3bKQA,5.0,eric best im la needed gel mani pedi eric help...,1,U1bmnQo9aGTORC-dzFA42Q,eric best im la needed gel mani pedi eric help...,eric best im la need gel mani pedi eric help d...,eric best im la need gel mani pedi eric help d...,eric best im la need gel mani pedi eric help d...
2,2,ZCsvwa_dsqNB8d8tYHPdjw,1,2009-03-14 21:11:47,1,NM4Fwh7VPINaiLA5bzbU-w,3.0,hate lesser rating review sir edmonds maybe do...,3,ZVu9TDpTvIgCN8x6IY-KmA,hate lesser rating review sir edmonds maybe do...,hate lesser rate review sir edmond mayb dozen ...,hate lesser rate review sir edmond mayb dozen ...,hate lesser rate review sir edmond mayb dozen ...
3,3,Ql5npMMRWKurN99tgA-g5w,0,2009-08-14 01:21:43,0,kb5Tl_Yyu919u7SzPaqQ0g,3.0,well staying 812 81409room nice upon looking g...,1,Dyn0ltgSpiCW3W2jqMVmlA,well staying 812 81409room nice upon looking g...,well stay 812 81409room nice upon look glass s...,well stay 812 81409room nice upon look glass s...,well stay room nice upon look glass shelf main...
4,4,WRQ-9LluyivReFiQZFUujw,3,2014-08-27 21:27:25,2,YaW20QCEhXNTOrvJtADJuw,5.0,fast food place make great hot dog different f...,3,dXIO-WFQJ8pwQ5M2STewZA,fast food place make great hot dog different f...,fast food place make great hot dog differ flav...,fast food place make great hot dog differ flav...,fast food place make great hot dog differ flav...


In [0]:
data_df = pd.DataFrame([])
data_df['x'] = df_all['text_clean'].apply(str)
data_df['y'] = df_all.apply(lambda x: str(int(x.stars) - 1), axis=1)


In [0]:
data_df.head(10)

,x,y
0,liter wouldnt abl find one neg thing say dr ma...,4
1,eric best im la need gel mani pedi eric help d...,4
2,hate lesser rate review sir edmond mayb dozen ...,2
3,well stay room nice upon look glass shelf main...,2
4,fast food place make great hot dog differ flav...,4
5,bakeri sey bet sweet buttercream cani ventur o...,3
6,novelti legoland attract open toronto area cer...,1
7,far biggest piec crap hotel ever servic slow b...,0
8,coloni vietnames great place busi luncha huge ...,3
9,favorit chipotl locat howev find order pick st...,3


In [0]:
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data_df['x'].values)
word_index = tokenizer.word_index

word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  
word_index["<UNUSED>"] = 3

tokenizer.word_index = word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
data_df['X'] = [x for x in X]
data_df['Y'] = [y for y in Y]

In [0]:
X = tokenizer.texts_to_sequences(data_df['x'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = data_df['y'].values
print('Shape of label tensor:', Y.shape)

Shape of data tensor: (1000000, 250)
Shape of label tensor: (1000000,)


In [0]:
tokenizer.sequences_to_texts([[1]])

['place']

In [0]:
Y = to_categorical(Y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [0]:
embedding_dim = 64
categoty_dim = len(Y[0])
vocab_size = MAX_NB_WORDS

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=MAX_SEQUENCE_LENGTH),
    layers.GlobalAveragePooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(categoty_dim, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 64)           640000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 648,965
Trainable params: 648,965
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=512, callbacks=[callback], validation_split=0.2)

W0822 20:18:09.002000 140219588122496 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 536000 samples, validate on 134000 samples
Epoch 1/30
536000/536000 [==============================] - 27s 51us/sample - loss: 0.9542 - accuracy: 0.6063 - val_loss: 0.8041 - val_accuracy: 0.6640
Epoch 2/30
536000/536000 [==============================] - 26s 49us/sample - loss: 0.7845 - accuracy: 0.6699 - val_loss: 0.7896 - val_accuracy: 0.6676
Epoch 3/30
536000/536000 [==============================] - 26s 49us/sample - loss: 0.7705 - accuracy: 0.6751 - val_loss: 0.7840 - val_accuracy: 0.6691
Epoch 4/30
536000/536000 [==============================] - 26s 49us/sample - loss: 0.7632 - accuracy: 0.6779 - val_loss: 0.7813 - val_accuracy: 0.6705
Epoch 5/30
536000/536000 [==============================] - 26s 49us/sample - loss: 0.7567 - accuracy: 0.6805 - val_loss: 0.7773 - val_accuracy: 0.6722
Epoch 6/30
536000/536000 [==============================] - 26s 49us/sample - loss: 0.7498 - accuracy: 0.6840 - val_loss: 0.7751 - val_accuracy: 0.6731
Epoch 7/30
536000/536000 [==========

In [0]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(acc)
print(val_acc)
plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) 

(10000, 64)


In [0]:
len(tokenizer.word_index)

580165

In [0]:
for word_num in range(2):
    word = decode_review([word_num])
    embeddings = weights[word_num]
    print(word)
    print(embeddings)

<PAD>
[ 3.08487732e-02 -2.14942116e-02 -7.30865262e-03 -1.79397166e-02
 -2.50233861e-04  2.85194861e-03  1.61988451e-03  2.72491723e-02
 -1.59611702e-02  1.92714890e-03  2.98261493e-02 -1.82294007e-02
  4.06449251e-02 -6.73580635e-03 -1.80583680e-03  2.97009274e-02
  2.19542533e-02 -5.65636996e-03  5.88051043e-05  1.87239442e-02
 -1.11676361e-02  1.81353744e-02 -2.46090740e-02 -1.08185485e-02
 -3.02369129e-02 -1.83733162e-02 -3.51357623e-03  2.95726936e-02
  1.17655676e-02  3.21063213e-02 -4.52863518e-03 -2.98990458e-02
 -2.28876043e-02 -1.62423011e-02  2.13906765e-02 -3.12791988e-02
  2.53675282e-02 -2.86115538e-02  1.58105802e-03  2.68167723e-02
 -2.12304275e-02 -6.58813445e-03 -1.07218660e-02 -1.09538285e-03
  2.18073698e-03 -8.17233871e-04  1.54988244e-02  3.90542224e-02
 -7.90382829e-03 -3.99258127e-03 -2.90282331e-02  3.05142961e-02
  6.75792154e-03 -1.65091567e-02 -1.62716247e-02  2.74020503e-03
  3.79621871e-02  4.31279698e-03 -1.24085210e-02 -2.79991478e-02
  2.63660699e-02 -2

In [0]:
data_df[['X', 'Y']].to_pickle("drive/My Drive/word2vec/data_vectorized.pickle")

In [0]:
a = pd.read_pickle("drive/My Drive/word2vec/data_vectorized.pickle")

In [0]:
with open('drive/My Drive/word2vec/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('drive/My Drive/word2vec/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [0]:
import io

In [0]:
np.savetxt("drive/My Drive/word2vec/embedding_weights.txt", weights)

In [0]:
weights = np.loadtxt("drive/My Drive/word2vec/embedding_weights.txt")
weights

array([[ 3.08487732e-02, -2.14942116e-02, -7.30865262e-03, ...,
        -2.33923234e-02, -3.82678807e-02, -1.13815013e-02],
       [ 8.40801001e-03,  1.29319318e-02,  4.72509898e-02, ...,
        -3.11039388e-04,  5.96653670e-04,  2.36457586e-03],
       [-1.30274147e-03, -1.52882561e-02, -4.59729917e-02, ...,
         6.82564825e-03,  4.04343493e-02, -3.36878188e-02],
       ...,
       [ 1.64468780e-01, -1.35418892e-01, -1.18391640e-01, ...,
        -5.36593087e-02, -2.52245486e-01, -9.96467546e-02],
       [-4.50488657e-01, -6.22870065e-02, -1.10623036e-02, ...,
         2.36492530e-02, -1.43581361e-01,  3.36197764e-02],
       [-6.68856427e-02, -1.01810165e-01, -2.32698068e-01, ...,
        -1.25630215e-01, -2.63715945e-02,  2.60064274e-01]])

In [0]:
weights.shape

(10000, 64)

In [0]:
import io

out_v = io.open("drive/My Drive/word2vec/vecs.tsv", 'w', encoding='utf-8')
out_m = io.open("drive/My Drive/word2vec/meta.tsv", 'w', encoding='utf-8')
for word_num in range(vocab_size-4):
    word = decode_review([word_num])
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
from google.colab import files
print(1)
files.download("drive/My Drive/word2vec/vecs.tsv")
print(2)
files.download("drive/My Drive/word2vec/meta.tsv")


1
2
